![title](ZLC.png)

 #  <font color='#B30838'> MZ 545 Supply Chain Network Design </font>

<div class="alert alert-info">  </h4> **Part 1: Single Facility Continuous Min-sum Location Models.**
</h4> </div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import googlemaps
import folium
from folium import features
from geopy import distance
from scipy.optimize import minimize

In [2]:
excel=pd.ExcelFile('CleanData-1.xlsx')
Data1 =pd.read_excel(excel,'Data')
display(Data1.head())
Data1.shape

,Market,City,TotalWeight (kg),Country,ZipCode,Latitude,Longitude
0,Market Austria,HORNSTEIN,173.856000,AT,7053,47.87925,16.44223
1,Market Austria,KLAGENFURT,4.000000,AT,9020,46.61670,14.30000
2,Market Austria,KLOSTERNEUBURG,789.835444,AT,3400,48.30420,16.31670
3,Market Austria,SILLIAN,2.000000,AT,9920,46.75280,12.42110
4,Market Belgium,AALST,376.311000,BE,9300,50.93330,4.03330


(2656, 7)

In [3]:
Data1.isnull().values.any()

False

In [4]:
# Sumary Market and Cities
Data1['City'].groupby([Data1['Market'],Data1['Country']]).count()

Market                Country
Market Austria        AT           9
Market Belgium        BE          84
                      LU           3
Market France         FR         661
Market Germany        DE         155
Market Italy          IT          27
Market Netherlands    NL         327
Market Spain & Portu  AD           1
                      ES         441
                      PT           9
Market Switzerland    CH          33
Market United Kingdo  GB         899
                      IE           7
Name: City, dtype: int64

In [5]:
color={'Market Austria':'#9f00a7','Market Belgium':'#2d89ef','Market France':'#2b5797','Market Germany':'#ffc40d',
       'Market Italy':'#00a300','Market Netherlands':'#da532c','Market Spain & Portu':'#99b433',
       'Market Switzerland':'#ff0097','Market United Kingdo':'#ee1111'}

$Min\;Z=\displaystyle\sum_{\forall \;k \in K} w_{k}d_{xy}$

In [6]:
weight={n:float(Data1.iloc[n][2])/Data1['TotalWeight (kg)'].sum() for n in range(len(Data1))}
customer=Data1['City'].index.values
Lat=Data1['Latitude'].values
Lon=Data1['Longitude'].values

In [7]:
def Euclidean(x):
    total=0
    for n in range(len(customer)):
        total=weight[n]*distance.distance((Lat[n],Lon[n]),(x[0],x[1])).km+total
    return total

In [8]:
def Rectilinear(x):
    total=0
    for n in range(len(customer)):
        total=weight[n]*(111*abs(Lat[n]-x[0])+111*abs(Lon[n]-x[1]))+total
    return total

In [9]:
def SQRT_Euclidean(x):
    total=0
    for n in range(len(customer)):
        factor1=(Lat[n]-x[0])^2
        factor2=(Lon[n]-x[1])^2
        total=weight[n]*(((111^2)*factor1+(111^2)*factor2)^0.5) + total
    return total

In [10]:
x0=[45.05,1.05]

In [11]:
sol = minimize(Rectilinear,x0, method='Nelder-Mead',)
Rectilinear_solution=sol.x
print(Rectilinear_solution[0])
print(Rectilinear_solution[1])

48.74016762
1.72702723844


In [12]:
sol = minimize(Euclidean,x0, method='Nelder-Mead',)
Euclidean_solution=sol.x
print(Euclidean_solution[0])
print(Euclidean_solution[1])

48.8662807195
2.27435613661


In [13]:
mapa = folium.Map(location=[Data1.iloc[0][5],Data1.iloc[0][6]],zoom_start=4)

fg=folium.FeatureGroup()  
for n in range(len(Data1)): 
    fg.add_child(folium.CircleMarker(location=[Data1.iloc[n][5],Data1.iloc[n][6]],radius=2,
                                     color=color[Data1.iloc[n][0]],fill = True,fill_color=color[Data1.iloc[n][0]]))

Dato='Rectilinear'
fg.add_child(folium.Marker(location=[Rectilinear_solution[0], Rectilinear_solution[1]],
                               popup=folium.Popup(Dato),
                               icon=folium.Icon(color='blue',
                                                icon_color='white',icon='info-sign')
                              ))

Dato='Euclidean'
fg.add_child(folium.Marker(location=[Euclidean_solution[0],Euclidean_solution[1]],
                               popup=folium.Popup(Dato),
                               icon=folium.Icon(color='green',
                                                icon_color='white',icon='info-sign')
                              ))


Dato='Sqr Euclidean'
fg.add_child(folium.Marker(location=[47.35774, 1.45758],
                               popup=folium.Popup(Dato),
                               icon=folium.Icon(color='red',
                                                icon_color='white',icon='info-sign')
                              ))
mapa.add_child(fg)
mapa